In [1]:
#importanto libs
import sqlite3
import pandas as pd
from datetime import datetime

# Criação do banco de dados e das tabelas
conn = sqlite3.connect('api/feature_store.db')
cursor = conn.cursor()

In [33]:
# Tabela para armazenar features de consumers
cursor.execute('''CREATE TABLE IF NOT EXISTS consumer_features (
                    customer_id TEXT PRIMARY KEY,
                    language TEXT,
                    created_at TIMESTAMP,
                    active INTEGER,
                    customer_phone_area TEXT,
                    customer_phone_number TEXT
                 )''')

In [34]:
# Tabela para armazenar features de restaurants
cursor.execute('''CREATE TABLE IF NOT EXISTS restaurant_features (
                    id TEXT PRIMARY KEY,
                    created_at TIMESTAMP,
                    enabled INTEGER,
                    price_range REAL,
                    average_ticket REAL,
                    takeout_time REAL,
                    delivery_time REAL,
                    minimum_order_value REAL,
                    merchant_zip_code TEXT,
                    merchant_city TEXT,
                    merchant_state TEXT,
                    merchant_country TEXT
                 )''')

In [35]:
conn.commit()

In [36]:
#ingestao de dados a partir do aquivo consumer.csv
def ingest_consumer_data(file_path='data/consumer.csv'):
    # Carrega o arquivo CSV
    df = pd.read_csv(file_path)
    
    # Conversão de tipos e manipulações, se necessário
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['active'] = df['active'].astype(int)  # Garantir que `active` seja binário
    
    # Inserção dos dados na tabela consumer_features
    df.to_sql('consumer_features', conn, if_exists='replace', index=False)
    print("Dados de consumer inseridos na Feature Store.")


In [37]:
#ingestao de dados a partir do aquivo consumer.csv
def ingest_restaurant_data(file_path='data/restaurant.csv'):
    # Carrega o arquivo CSV
    df = pd.read_csv(file_path)
    
    # Conversão de tipos e manipulações, se necessário
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['enabled'] = df['enabled'].astype(int)

    # Inserção dos dados na tabela restaurant_features
    df.to_sql('restaurant_features', conn, if_exists='replace', index=False)
    print("Dados de restaurant inseridos na Feature Store.")

In [38]:
#Executando as Funções para Ingestão
ingest_consumer_data()  # Carrega dados do arquivo consumer.csv
ingest_restaurant_data()  # Carrega dados do arquivo restaurant.csv

Dados de consumer inseridos na Feature Store.
Dados de restaurant inseridos na Feature Store.


In [39]:
# Consultando os dados da tabela consumer
consumer_data = pd.read_sql_query("SELECT * FROM consumer_features LIMIT 5", conn)
print("Consumer data:", consumer_data)

Consumer data:                                          customer_id language  \
0  e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e...    pt-br   
1  a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d...    pt-br   
2  41e1051728eba13341136d67d0757f8d8cc44b2a405b71...    pt-br   
3  8e7c1dcb64edf95c935147f6d560cb068c44714cb1b21b...    pt-br   
4  7823d4cf4150c5daeae0bd799206852fc123bdef0cf5d1...    pt-br   

                         created_at  active customer_name  \
0  2018-04-05 14:49:18.165000+00:00       1          NUNO   
1  2018-01-14 21:40:02.141000+00:00       1      ADRIELLY   
2  2018-01-07 03:47:15.554000+00:00       1         PAULA   
3  2018-01-10 22:17:08.160000+00:00       1        HELTON   
4  2018-04-06 00:16:20.935000+00:00       1        WENDER   

   customer_phone_area  customer_phone_number  
0                   46              816135924  
1                   59              231330577  
2                   62              347597883  
3                   13             

In [40]:
# Consultando os dados da tabela restaurant
restaurant_data = pd.read_sql_query("SELECT * FROM restaurant_features LIMIT 5", conn)
print("Restaurant data:", restaurant_data)

Restaurant data:                                                   id  \
0  d19ff6fca6288939bff073ad0a119d25c0365c407e9e5d...   
1  631df0985fdbbaf27b9b031a8f381924e3483833385748...   
2  135c5c4ae4c1ec1fdc23e8c649f313e39be8db913d8bc5...   
3  d26f84c470451f752bef036c55517b6d9950d41806f10e...   
4  97b9884600ea7192314580d9115f8882b8634f5aa201ff...   

                         created_at  enabled  price_range  average_ticket  \
0  2017-01-23 12:52:30.910000+00:00        0            3            60.0   
1  2017-01-20 13:14:48.286000+00:00        1            3            60.0   
2  2017-01-23 12:46:33.457000+00:00        1            5           100.0   
3  2017-01-20 13:15:04.806000+00:00        1            3            80.0   
4  2017-01-20 13:14:27.701000+00:00        1            3            60.0   

   takeout_time  delivery_time  minimum_order_value  merchant_zip_code  \
0             0             50                 30.0              14025   
1             0              0     

In [41]:
#função para cada conjunto de dados (consumer e restaurant) para ingestão e pré-processamento
def ingest_consumer_data(file_path):
    df = pd.read_csv(file_path)
    
    # Conversão de tipos e manipulações
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['active'] = df['active'].astype(int)  # Garantir que `active` seja binário

    # Inserção no banco de dados
    df.to_sql('consumer_features', conn, if_exists='replace', index=False)
    print("Dados de consumer ingeridos na Feature Store.")


In [42]:
#função para cada conjunto de dados (consumer e restaurant) para ingestão e pré-processamento
def ingest_restaurant_data(file_path):
    df = pd.read_csv(file_path)
    
    # Conversão de tipos e manipulações
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['enabled'] = df['enabled'].astype(int)

    # Inserção no banco de dados
    df.to_sql('restaurant_features', conn, if_exists='replace', index=False)
    print("Dados de restaurant ingeridos na Feature Store.")

In [43]:
#Funções para Extração de Features
#Essas funções extraem e retornam as features necessárias para treinar modelos específicos
def get_consumer_features():
    query = "SELECT * FROM consumer_features"
    return pd.read_sql_query(query, conn)

def get_restaurant_features():
    query = "SELECT * FROM restaurant_features"
    return pd.read_sql_query(query, conn)

# Exemplo de função que combina features dos dois datasets
def get_combined_features():
    query = '''
    SELECT c.customer_id, r.id, c.language, c.active, r.price_range, r.average_ticket
    FROM consumer_features AS c
    INNER JOIN restaurant_features AS r
    ON c.customer_id = r.id  -- Exemplo de join, adapte conforme a relação real
    '''
    return pd.read_sql_query(query, conn)

In [44]:
#Pipeline de Treinamento Usando as Features da Feature Store
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [45]:
# Exemplo para treinamento com dados de consumidores
def train_consumer_model():
    df = get_consumer_features()
    X = df.drop(columns=['customer_id', 'customer_name'])  # Colunas irrelevantes
    y = df['active']  # Alvo

    # Divisão treino/teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Modelo e treinamento
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Avaliação
    y_pred = model.predict(X_test)
    print(f"Acurácia do modelo de consumidor: {accuracy_score(y_test, y_pred):.4f}")

In [46]:
# Exemplo para treinamento com dados de restaurantes
def train_restaurant_model():
    df = get_restaurant_features()
    X = df.drop(columns=['id', 'created_at', 'merchant_zip_code'])  # Colunas irrelevantes
    y = df['enabled']  # Alvo

    # Divisão treino/teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Modelo e treinamento
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Avaliação
    y_pred = model.predict(X_test)
    print(f"Acurácia do modelo de restaurante: {accuracy_score(y_test, y_pred):.4f}")

In [31]:
#Interagindo com o modelo através de uma API
#necessarios os pacotes fastapi e uvicorn (server)

#Arquivos base da API
#1.main.py - possui definicao dos endpoints
#2.crud.py - possui funcoes crud para criar, ler, atualizar e deletar dados do banco
#3.models.py - definicao dos modelos
#4.database.py - conexao com banco de dados

In [35]:
#Executando a api
#executar o comando na terminal
# necessário estar no diretorio raiz do projeto /featurestore/api
# --> uvicorn main:app --reload

#Após executar o comando no terminal
#a aplicação carregará na porta 8000 por padrão: http://127.0.0.1:8000

#Endpoints Disponíveis:

#POST /consumers/: Insere um novo consumidor.
#GET /consumers/: Retorna todos os consumidores.
#POST /restaurants/: Insere um novo restaurante.
#GET /restaurants/: Retorna todos os restaurantes.

#Documentação disponível em http://127.0.0.1:8000/docs

#Testando os Endpoints
#Utilize ferramentas como Postman/SoapUI 
# ou a interface do Swagger (http://127.0.0.1:8000/docs) 
# para testar os endpoints 
# e verificar a comunicação com a feature store no banco SQLite